In [ ]:
import math

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

In [ ]:
# The path to a gzipped CSV containing forex data from Polygon.io.
DATA_PATH = "../../data/raw/2025-03-03.csv.gz"

# The currencies in which we are interested.
# Most traded currencies -> https://en.wikipedia.org/wiki/Template:Most_traded_currencies.
# CURRENCIES = {"CHF", "ZAR", "CAD", "AUD", "USD", "SGD", "EUR", "GBP", "SEK", "JPY", "DKK", "HKD", "NOK", "MXN", "NZD", "CZK", "PLN", "HUF", "TRY", "TWD", "RUB", "INR", "CLP", "CNY", "CNH", "THB", "BGN", "MYR", "KRW", "ILS"}
# CURRENCIES = {"USD", "SGD", "EUR", "GBP", "JPY"}
CURRENCIES = {"USD", "EUR", "JPY", "GBP", "CNH", "AUD", "CAD", "CHF", "HKD", "SGD"}

In [ ]:
df = pd.read_csv(DATA_PATH, compression="gzip")

df

In [ ]:
# Clean, filter, and sort the raw dataframe.

def is_symbol_of_interest(symbol: str):
    currency1, currency2 = symbol[2:].split("-")
    return currency1 in CURRENCIES and currency2 in CURRENCIES

# Sort by the timestamp and convert the timestamp to seconds.
# The timestamp is provided in nanoseconds but has 1 second resolution.
df = df.sort_values(by="participant_timestamp", kind="stable")
df["participant_timestamp"] //= int(1e9)

# The exchange always seems to equal 48.
df = df.drop(["ask_exchange", "bid_exchange"], axis=1)

# Remove currencies in which we are not interested.
df = df[df["ticker"].map(is_symbol_of_interest)]

# OPTIONAL
# Drop duplicate symbols within the same timestamp.
# When this happens, we should keep the last (most up-to-date) prices.
df = df.drop_duplicates(subset=["ticker", "participant_timestamp"], keep="last")

df

In [ ]:
df.to_csv(DATA_PATH.replace(".csv.gz", "_filtered.csv"))

In [ ]:
# Construct a directed, weighted graph in which the nodes
# are currencies and the edge weights are exchange rates.

graph = dict()

for index, row in df.iterrows():
    currency1, currency2 = row["ticker"][2:].split("-")
    ask_price = row["ask_price"]

    if currency1 not in graph:
        graph[currency1] = dict()
    graph[currency1][currency2] = ask_price

In [ ]:
# Visualize the forex graph.

G = nx.DiGraph()

for currency1, rates in graph.items():
    for currency2, rate in rates.items():
        G.add_edge(currency1, currency2, weight=rate)

pos = nx.nx_pydot.graphviz_layout(G, prog="neato")
edge_labels = nx.get_edge_attributes(G, "weight")

plt.figure(figsize=(5, 5))
nx.draw(G, pos, with_labels=True, node_size=1000, node_color="#FF7777", connectionstyle="arc3,rad=0.5")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, connectionstyle="arc3,rad=0.5")

plt.savefig("graph.png", dpi=300)
plt.show()

In [ ]:
# Simple functions to help perform arbitrage.

def graph_exchange_to_log(graph: dict[str, dict[str, float]]) -> dict[str, dict[str, float]]:
    log_graph = dict()
    for currency in graph:
        log_graph[currency] = {c: -math.log(w) for c, w in graph[currency].items()}
    return log_graph

def bellman_ford(graph: dict[str, dict[str, float]]) -> bool:
    # Create a new graph in which each weight is the negative logarithm of the exchange rate.
    graph = graph_exchange_to_log(graph)

    # Add a sentinel source node with a weight 0 edge to each node.
    graph["\0"] = {c: 0 for c in graph}

    # Initialize the Bellman-Ford data structures.
    distance = {c: float("inf") for c in graph}
    distance["\0"] = 0
    predecessor = {c: None for c in graph}

    # Relax the edges V - 1 times.
    for _ in range(len(graph) - 1):
        for c1 in graph:
            for c2 in graph[c1]:
                weight = graph[c1][c2]
                if distance[c1] + weight < distance[c2]:
                    distance[c2] = distance[c1] + weight
                    predecessor[c2] = c1

    # Try to relax edges one more time. If possible, then there is a negative cycle.
    skips = 0
    for c1 in graph:
        for c2 in graph[c1]:
            weight = graph[c1][c2]
            if distance[c1] + weight < distance[c2]:
                if skips > 0:
                    skips -= 1
                    continue
                predecessor[c2] = c1
                visited = {c: False for c in graph}
                visited[c2] = True
                while not visited[c1]:
                    visited[c1] = True
                    c1 = predecessor[c1]
                negative_cycle = [c1]
                c2 = predecessor[c1]
                while c1 != c2:
                    negative_cycle = [c2] + negative_cycle
                    c2 = predecessor[c2]
                return negative_cycle

    return False

result = bellman_ford(graph)
if result:
    print(" -> ".join(result) + " -> " + result[0])